Project Goal

This project aims to analyze Medicaid drug utilization data to identify key spending drivers and build a predictive framework for reimbursement analysis.

Specifically, we will develop:
A Diagnostic Analysis: 
To decompose spending increases into Price vs. Volume effects.

An Anomaly Detection Model: 
Using regression techniques to identify claims where reimbursement amounts deviate significantly from expected patterns (potential overpayment).

A 'Budget Impact' Web Tool:
A simulation dashboard that allows policymakers to forecast Q3/Q4 spending based on user-defined growth scenarios (Best/Worst case)."
 
4. "Generic Substitution Analysis."
The Logic: Find expensive "Brand Name" drugs in your data that have a cheaper "Generic" equivalent.
The Calculation:(Price of Brand - Price of Generic) $\times$ Brand Volume = Potential Savings.
The Insight: "We could save the state $5 Million just by switching these 3 drugs to generic."

Import the necessary Libraries

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Load The Data

In [4]:
df = pd.read_csv('Medicaid_data (1).csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313397 entries, 0 to 1313396
Data columns (total 15 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   Utilization Type                1313397 non-null  object 
 1   State                           1313397 non-null  object 
 2   NDC                             1313397 non-null  int64  
 3   Labeler Code                    1313397 non-null  int64  
 4   Product Code                    1313397 non-null  int64  
 5   Package Size                    1313397 non-null  int64  
 6   Year                            1313397 non-null  int64  
 7   Quarter                         1313397 non-null  int64  
 8   Suppression Used                1313397 non-null  bool   
 9   Product Name                    1313397 non-null  object 
 10  Units Reimbursed                1313397 non-null  float64
 11  Number of Prescriptions         1313397 non-null  float64
 12  

In [5]:
df.head(15)

,Utilization Type,State,NDC,Labeler Code,Product Code,Package Size,Year,Quarter,Suppression Used,Product Name,Units Reimbursed,Number of Prescriptions,Total Amount Reimbursed,Medicaid Amount Reimbursed,Non Medicaid Amount Reimbursed
0,FFSU,AK,2143380,2,1433,80,2025,2,False,TRULICITY,216.0,107.0,102976.40,98630.87,4345.53
1,FFSU,AK,2143480,2,1434,80,2025,2,False,TRULICITY,218.0,109.0,104481.92,101806.64,2675.28
2,FFSU,AK,2143611,2,1436,11,2025,2,False,EMGALITY P,21.0,20.0,15227.25,15227.25,0.00
3,FFSU,AK,2144511,2,1445,11,2025,2,False,TALTZ AUTO,33.0,30.0,231532.28,231532.28,0.00
4,FFSU,AK,2145780,2,1457,80,2025,2,False,MOUNJARO,208.0,104.0,108908.80,105953.32,2955.48
5,FFSU,AK,2146080,2,1460,80,2025,2,False,MOUNJARO,120.0,60.0,62499.89,54631.25,7868.64
6,FFSU,AK,2147180,2,1471,80,2025,2,False,MOUNJARO,170.0,85.0,89214.16,82678.09,6536.07
7,FFSU,AK,2148480,2,1484,80,2025,2,False,MOUNJARO,198.0,97.0,103266.58,102345.52,921.06
8,FFSU,AK,2149580,2,1495,80,2025,2,False,MOUNJARO,178.0,89.0,93349.46,89341.68,4007.78
9,FFSU,AK,2150680,2,1506,80,2025,2,False,MOUNJARO,96.0,48.0,50615.54,50098.40,517.14


Check the number of drugs in the dataset

In [11]:
productname_unique = df['Product Name'].value_counts()
productname_unique

Product Name
LEVOTHYROX    17310
GABAPENTIN    13730
METOPROLOL    13721
ATORVASTAT    13614
QUETIAPINE    12314
              ...  
DIVIGEL  E        1
CORIFACT          1
KCENTRA 40        1
JOENJA            1
DUREZOL 0.        1
Name: count, Length: 10540, dtype: int64

Check for suppression == True to know whether you need to filter

In [12]:
Suppression_True = df['Suppression Used'].value_counts()
Suppression_True

Suppression Used
False    1313397
Name: count, dtype: int64

The suppression column showed where there are some prescriptions replaced by zero's due to very little patients of that drug. 
The code above shows that there is no column where suppression == True

Check for the drugs that cost Medicaid a lot of money

In [13]:
top_drugs = df.groupby('Product Name')['Medicaid Amount Reimbursed'].sum().sort_values(ascending=False).head(50)
top_drugs

Product Name
HUMIRA PEN    2.221647e+09
TRULICITY     1.964051e+09
INVEGA SUS    1.750146e+09
DUPIXENT S    1.644891e+09
Biktarvy      1.609172e+09
BIKTARVY      1.461354e+09
Jardiance     1.203436e+09
JARDIANCE     1.203039e+09
VRAYLAR (C    1.160832e+09
ZEPBOUND      1.096850e+09
OZEMPIC 0.    1.079653e+09
STELARA 90    1.034360e+09
SKYRIZI 15    9.056571e+08
TRIKAFTA (    8.998348e+08
OZEMPIC 2M    8.906712e+08
HUMIRA(CF)    8.532094e+08
OZEMPIC 1M    8.334060e+08
ABILIFY MA    8.192711e+08
COSENTYX S    6.896762e+08
Eliquis       6.727761e+08
OZEMPIC       6.485346e+08
SUBLOCADE     6.474153e+08
COSENTYX U    6.384966e+08
ELIQUIS       5.982808e+08
DUPIXENT P    5.975172e+08
KEYTRUDA 1    5.914211e+08
ENBREL (et    5.423337e+08
TRELEGY EL    5.408418e+08
DUPIXENT 3    5.148154e+08
ENTRESTO F    4.919323e+08
WEGOVY 2.4    4.890721e+08
TALTZ         4.853750e+08
MAVYRET 10    4.803955e+08
NORDITROPI    4.797721e+08
INVEGA TRI    4.670399e+08
Suboxone S    4.082682e+08
MOUNJARO 1    3

Check for 80% of top spenders from tou data set

In [18]:
#calculate cumulative sum of top drugs
top_drugs_cumsum = top_drugs.cumsum()
top_drugs_cumsum

Product Name
HUMIRA PEN    2.221647e+09
TRULICITY     4.185697e+09
INVEGA SUS    5.935844e+09
DUPIXENT S    7.580735e+09
Biktarvy      9.189907e+09
BIKTARVY      1.065126e+10
Jardiance     1.185470e+10
JARDIANCE     1.305774e+10
VRAYLAR (C    1.421857e+10
ZEPBOUND      1.531542e+10
OZEMPIC 0.    1.639507e+10
STELARA 90    1.742943e+10
SKYRIZI 15    1.833509e+10
TRIKAFTA (    1.923492e+10
OZEMPIC 2M    2.012560e+10
HUMIRA(CF)    2.097880e+10
OZEMPIC 1M    2.181221e+10
ABILIFY MA    2.263148e+10
COSENTYX S    2.332116e+10
Eliquis       2.399393e+10
OZEMPIC       2.464247e+10
SUBLOCADE     2.528988e+10
COSENTYX U    2.592838e+10
ELIQUIS       2.652666e+10
DUPIXENT P    2.712418e+10
KEYTRUDA 1    2.771560e+10
ENBREL (et    2.825793e+10
TRELEGY EL    2.879878e+10
DUPIXENT 3    2.931359e+10
ENTRESTO F    2.980552e+10
WEGOVY 2.4    3.029460e+10
TALTZ         3.077997e+10
MAVYRET 10    3.126037e+10
NORDITROPI    3.174014e+10
INVEGA TRI    3.220718e+10
Suboxone S    3.261545e+10
MOUNJARO 1    3

From this output, we see that our data has duplicate names for the drugs some in lower case and other in upper case but they are the same drug, so we make everything uppercase in that column.

In [28]:
df['Product Name Clean'] = df['Product Name'].str.upper()

Combine dosages since a drug like ozempic is stil split

In [29]:
df['Product Name Clean'] = df['Product Name Clean'].str.split().str[0]

In [30]:
top_drugs_clean = df.groupby('Product Name Clean')['Medicaid Amount Reimbursed'].sum().sort_values(ascending=False)
top_drugs_clean.head(50)

Product Name Clean
OZEMPIC       3.583372e+09
BIKTARVY      3.381264e+09
DUPIXENT      2.800248e+09
HUMIRA        2.743383e+09
JARDIANCE     2.406476e+09
INVEGA        2.305583e+09
WEGOVY        2.117002e+09
TRULICITY     1.964051e+09
VRAYLAR       1.569739e+09
SKYRIZI       1.545642e+09
STELARA       1.475403e+09
COSENTYX      1.347543e+09
ELIQUIS       1.345077e+09
TRIKAFTA      1.332607e+09
MOUNJARO      1.207198e+09
ZEPBOUND      1.194849e+09
ENBREL        1.076897e+09
VYVANSE       1.056195e+09
ABILIFY       9.812919e+08
HEMLIBRA      8.810908e+08
HUMIRA(CF)    8.532094e+08
FARXIGA       8.445481e+08
SUBOXONE      8.174783e+08
RINVOQ        7.391217e+08
KEYTRUDA      7.171740e+08
SYMBICORT     6.765533e+08
ENTRESTO      6.635851e+08
SUBLOCADE     6.474153e+08
INGREZZA      6.153266e+08
TALTZ         5.980681e+08
NURTEC        5.967303e+08
MAVYRET       5.857936e+08
FLUTICASON    5.684319e+08
LINZESS       5.553370e+08
TRELEGY       5.408418e+08
EPIDIOLEX     5.046786e+08
REXULTI  

In [36]:
print(len(top_drugs_clean))

3919


In [37]:
top_drugs_clean.shape

(3919,)

In [31]:
cumulative_sum_clean = top_drugs_clean.cumsum()
cumulative_sum_clean.head(50)

Product Name Clean
OZEMPIC       3.583372e+09
BIKTARVY      6.964636e+09
DUPIXENT      9.764884e+09
HUMIRA        1.250827e+10
JARDIANCE     1.491474e+10
INVEGA        1.722033e+10
WEGOVY        1.933733e+10
TRULICITY     2.130138e+10
VRAYLAR       2.287112e+10
SKYRIZI       2.441676e+10
STELARA       2.589216e+10
COSENTYX      2.723971e+10
ELIQUIS       2.858478e+10
TRIKAFTA      2.991739e+10
MOUNJARO      3.112459e+10
ZEPBOUND      3.231944e+10
ENBREL        3.339633e+10
VYVANSE       3.445253e+10
ABILIFY       3.543382e+10
HEMLIBRA      3.631491e+10
HUMIRA(CF)    3.716812e+10
FARXIGA       3.801267e+10
SUBOXONE      3.883015e+10
RINVOQ        3.956927e+10
KEYTRUDA      4.028644e+10
SYMBICORT     4.096300e+10
ENTRESTO      4.162658e+10
SUBLOCADE     4.227400e+10
INGREZZA      4.288932e+10
TALTZ         4.348739e+10
NURTEC        4.408412e+10
MAVYRET       4.466991e+10
FLUTICASON    4.523835e+10
LINZESS       4.579368e+10
TRELEGY       4.633453e+10
EPIDIOLEX     4.683920e+10
REXULTI  

In [32]:
total_medicaid_spend = df['Medicaid Amount Reimbursed'].sum()
total_medicaid_spend

104001893480.23996

In [33]:
cumulative_percentage = (cumulative_sum_clean / total_medicaid_spend) * 100

In [34]:
top_80_percent = cumulative_percentage[cumulative_percentage <= 80]
top_80_percent

Product Name Clean
OZEMPIC        3.445487
BIKTARVY       6.696643
DUPIXENT       9.389141
HUMIRA        12.026961
JARDIANCE     14.340837
                ...    
SKYCLARYS     79.709436
IMCIVREE      79.775782
LOSARTAN      79.841705
DEXMETHYLP    79.907598
QVAR          79.973373
Name: Medicaid Amount Reimbursed, Length: 248, dtype: float64

In [43]:
df.shape

(1313397, 17)

Check what cause the price increase, volume or price.

In [41]:
df['price_check'] = df.groupby('Product Name')['Medicaid Amount Reimbursed'].sum().sort_values(ascending=False).reset_index()
df['price_check'].head(50)

ValueError: Columns must be same length as key